## Описание проекта

Необходимо защитить данные клиентов страховой компании «Хоть потоп». Для этого нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

## 1. Изучение данных из файла

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
try:
    df = pd.read_csv("C:/Users/dimil/OneDrive/Desktop/insurance.csv")
except:
    df = pd.read_csv('/datasets/insurance.csv')

df.head(10)

In [3]:
df.describe().T

In [4]:
df.info()

Первое знакомство с данными показывает отсутствие каких-либо пропусков или аномалий.

## 2. Умножение матриц

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$
    
Предсказания на исходных признаках: $a = Xw$. Предсказания на преобразованных признаках: $a_1 = X_1 w_1$. $X_1 = XP$, где $P$ это матрица шифрования.
    
Докажем, что $a = a_1$. Для этого нужно преобразовать выражения. 
    
    
$w_1 = (X_1^T X_1)^{-1} X_1^T y = ((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} X^T y$ 

$a_1 = X_1 P^{-1} (X^T X)^{-1} X^T y = XP P^{-1} (X^T X)^{-1} X^T y = Xw = a$

Следовательно, $a = Xw = a1$, что и требовалось доказать.

## 3. Алгоритм преобразования данных для решения задачи.

Перед внесением возможных изменений в датасет, изучим признаки на наличие корреляции:

In [5]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

Похоже, что возраст - это единственный признак, по которому наблюдается существенная корреляция с целевым признаком.

## 4. Применение алгоритма и оценка качества линейной регрессии

Сначала построим регрессию, используя все доступные признаки:

In [6]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

Еще раз убедимся, что возраст является основным признаком, оказывающим влияние на целевой признак. Для этого построим регрессию по нему и любому другому признаку.

In [7]:
features_1 = df[['Возраст','Пол']]
target_1 = df['Страховые выплаты']

   
model = LinearRegression()
model.fit(features_1, target_1)
predictions_1 = model.predict(features_1)
print(r2_score(target_1, predictions_1))

Действительно, метрика R2 практически не изменилась. Теперь закодируем наши признаки так, чтобы по ним было сложно восстановить персональную информацию клиентов. После умножения на случайную обратиимую матрицу, метрика R2 останется на том же уровне. Проверим это:

In [8]:
encoder_matrix = np.random.normal(size=(4, 4))

In [9]:
features_encoded = features.dot(encoder_matrix)

In [10]:
model = LinearRegression()
model.fit(features_encoded, target)
predictions_encoded = model.predict(features_encoded)
print(r2_score(target, predictions_encoded))

Версия о неизменности R2 подтвердилась. Таким образом, данные будут надежно закодированы, а качество регрессии не изменится.